In [1]:
%%time
import pandas as pd
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
#from numpy import loadtxt
from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder


from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize

Wall time: 3.89 s


In [2]:
%%time
df = pd.read_excel("BQ Analysis v1_1.xlsx", "Project BQ")

Wall time: 10.4 s


In [3]:
%%time
df = df.astype(str)
df["PREPROCESS_TEXT"] = df["BQ_TRADE"] + "," + df["BQ_HEADING"] + "," + df["BQ_SUB_HEADING"] + "," + df["BQ_ITEM_HEADING"] + "," + df["BQ_ITEM_DESCRIPTION"] #+ "," + df["AC_TRADE_CODE"]
#df['RESULT'] = ""
#training_df = df.loc[df['CONTRACT_CODE'] == "FL49"].copy() // not enough training data?
df = df[df['AC_TRADE_CODE'] != "DA"]
df = df[df['AC_TRADE_CODE'] != "IP"]
df = df[df['AC_TRADE_CODE'] != "SS"]
df = df[df['AC_TRADE_CODE'] != "EL"]
df = df[df['AC_TRADE_CODE'] != "SO"]
df = df[df['AC_TRADE_CODE'] != "AS"]


df["PREPROCESS_TEXT"] = list(map(lambda x: x.lower(), list(df["PREPROCESS_TEXT"])))





Wall time: 358 ms


In [4]:
class_names = ['PS',
'ST',
'SM',
'EL',
'PL',
'BS',
'DR',
'CJ',
'PA',
'AR',
'PR',
'SO',
'GL',
'IF',
'DW',
'EX',
'IR',
'FD',
'AP',
'SU',
'FF',
'SG',
'AS',
'EF',
'BR',
'SS',
'DA',
'IP'
]





In [5]:
training_df = df[df['CONTRACT_CODE'] != "QHP3"].copy()


training_df["AC_TRADE_CODE_INDEX"] = list(map(lambda x: class_names.index(x), list(training_df.AC_TRADE_CODE)))
#training_df["AC_TRADE_CODE"].value_counts()
#training_df["AC_TRADE_CODE_INDEX"].value_counts()

In [6]:
test_df = df[df['CONTRACT_CODE'] == "QHP3"].copy()
test_df["AC_TRADE_CODE_INDEX"] = list(map(lambda x: class_names.index(x), list(test_df.AC_TRADE_CODE)))
test_df["AC_TRADE_CODE_INDEX"].value_counts()

0     2788
1     2712
2     2258
4      853
6      479
8      427
7      368
9      215
12      95
10      58
24      35
16      24
15      10
Name: AC_TRADE_CODE_INDEX, dtype: int64

In [7]:
# print(len(df["AC_TRADE_CODE"]))
# label_encoder = LabelEncoder()
# label_encoder.fit(df["AC_TRADE_CODE"])
# training_label = np_utils.to_categorical((label_encoder.transform(df["AC_TRADE_CODE"])))

# print(len(training_label))

In [8]:
# %%time
# tokenized_doc = []
# for d in training_df["PREPROCESS_TEXT"]:
#     tokenized_doc.append(word_tokenize(d))
# tokenized_doc
# tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]
# len(tagged_data)

In [9]:
%%time
model= Doc2Vec.load("test_doc2vec.model")
#training_text = model.wv.vectors
#print(len(training_text))

Wall time: 249 ms


In [10]:
%%time
# test_data = word_tokenize(df["PREPROCESS_TEXT"])
# v1 = model.infer_vector(test_data)
# print("V1_infer", v1)

training_text = []
for index, row in training_df.iterrows():
    training_text.append(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))

training_text = np.asarray(training_text)  
print(len(training_text))
print(training_text[0])





test_text = []
for index, row in test_df.iterrows():
    #print(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))
    test_text.append(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))

test_text = np.asarray(test_text)  
print(len(test_text))
print(test_text[0])


31075
[-0.07133482 -0.08574978 -0.41277137 -0.63597965 -0.91471535  0.28850505
 -0.53576374 -0.90945405 -1.5025458  -0.61843145 -0.48765257  0.3181048
 -0.10366034 -1.4194092   0.6623187   0.8691084  -0.16303876  0.806646
  0.8260437   1.0821548 ]
10322
[ 0.47780302  1.2907476  -1.1757203   0.54326457  0.48385    -1.4731838
 -0.08764178  0.01109768 -0.37486562 -0.18048786  0.98855644  0.6542719
  1.7492304   0.11882848  0.3736593  -0.6452582  -0.6972129   1.0269859
 -0.31154588  2.3005223 ]
Wall time: 6min 6s


In [11]:
training_label = np.asarray(training_df["AC_TRADE_CODE_INDEX"])  


In [12]:
training_text.shape, training_label.shape


((31075, 20), (31075,))

In [14]:

%%time
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, )),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(50)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(training_text, training_label, epochs=100)


Epoch 1/100
972/972 [==============================] - 1s 822us/step - loss: 1.4920 - accuracy: 0.6134
Epoch 2/100
972/972 [==============================] - 1s 798us/step - loss: 0.5886 - accuracy: 0.8239
Epoch 3/100
972/972 [==============================] - 1s 845us/step - loss: 0.4963 - accuracy: 0.8508
Epoch 4/100
972/972 [==============================] - 1s 899us/step - loss: 0.4447 - accuracy: 0.8626
Epoch 5/100
972/972 [==============================] - 1s 866us/step - loss: 0.4098 - accuracy: 0.8737
Epoch 6/100
972/972 [==============================] - 1s 828us/step - loss: 0.3812 - accuracy: 0.8835
Epoch 7/100
972/972 [==============================] - 1s 806us/step - loss: 0.3454 - accuracy: 0.8928
Epoch 8/100
972/972 [==============================] - 1s 911us/step - loss: 0.3388 - accuracy: 0.8990
Epoch 9/100
972/972 [==============================] - 1s 887us/step - loss: 0.3233 - accuracy: 0.9031
Epoch 10/100
972/972 [==============================] - 1s 818us/step - l

972/972 [==============================] - 1s 752us/step - loss: 0.1179 - accuracy: 0.9619
Epoch 80/100
972/972 [==============================] - 1s 733us/step - loss: 0.1113 - accuracy: 0.9651
Epoch 81/100
972/972 [==============================] - 1s 755us/step - loss: 0.1119 - accuracy: 0.9630
Epoch 82/100
972/972 [==============================] - 1s 740us/step - loss: 0.1133 - accuracy: 0.9648
Epoch 83/100
972/972 [==============================] - 1s 748us/step - loss: 0.1105 - accuracy: 0.9638
Epoch 84/100
972/972 [==============================] - 1s 748us/step - loss: 0.1077 - accuracy: 0.9653
Epoch 85/100
972/972 [==============================] - 1s 732us/step - loss: 0.1068 - accuracy: 0.9647
Epoch 86/100
972/972 [==============================] - 1s 738us/step - loss: 0.1072 - accuracy: 0.9649
Epoch 87/100
972/972 [==============================] - 1s 742us/step - loss: 0.1058 - accuracy: 0.9658
Epoch 88/100
972/972 [==============================] - 1s 732us/step - loss:

In [15]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_text)

In [16]:

i = 0
test_df["PREDICT_RESULT"] = 0
for index, row in test_df.iterrows():
    test_df["PREDICT_RESULT"][index] = np.argmax(predictions[i])
    i = i + 1


test_df["COMPARE"] = np.where(test_df["AC_TRADE_CODE_INDEX"] == test_df["PREDICT_RESULT"], True, False)


<ipython-input-16-7c42d552e199>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["PREDICT_RESULT"][index] = np.argmax(predictions[i])


In [17]:
test_df['COMPARE'].value_counts()

True     9328
False     994
Name: COMPARE, dtype: int64